# Example MNIST dataset
- In this notebook, we introduce how to use `ShapEngine` to compute Beta Shapley value. 

In [1]:
import sys, argparse, random
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"
os.environ["LD_LIBRARY_PATH"]="/home/wp2280/cuda_lib_temp/"

! echo $LD_LIBRARY_PATH

sys.path.append('../betashap')

from time import time
import numpy as np
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


from tensorflow import keras
np.random.seed(2022)
import utils, data
tf.config.run_functions_eagerly(True)

/home/wp2280/cuda_lib_temp/


2022-04-27 23:34:26.579537: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-27 23:34:26.681043: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/wp2280/cuda_lib_temp/
2022-04-27 23:34:26.682008: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registe

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17113836229084143116
xla_global_id: -1
]


# Get Original Dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# train_labels = train_labels[:1000]
# test_labels = test_labels[:1000]
#
# train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
# test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

train_labels = tf.one_hot(train_labels, 10)
test_labels = tf.one_hot(test_labels, 10)

train_images = tf.expand_dims(train_images, -1)
test_images = tf.expand_dims(test_images, -1)
train_images = tf.cast(train_images, tf.float32) / 255.0
test_images = tf.cast(test_images, tf.float32) / 255.0

2022-04-27 23:34:28.547317: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Compute Beta Shapley

In [3]:
class Args():
    def __init__(self):
        self.rate = 0.001
        self.epoch = 1
        self.seed = 1
        self.loss = ''
        self.checkpoint = ''
        self.path = 'checkpoint'
        self.activation = 'relu'
        self.save_checkpoint = False
args = Args()

In [4]:
def faultyloss(probs, labels):
    """Calculates cross entropy loss.
    Args:
      probs: Class probabilities predicted by the model. The shape is expected
        to be (?, 10).
      labels: Truth labels for the classes, as one-hot encoded vectors. The
        shape is expected to be the same as `probs`.
    Returns:
      A scalar loss tensor.
    """
    diff = -labels * tf.math.log(probs)
    loss = tf.reduce_mean(diff)
    return loss

def create_model(loss=tf.losses.CategoricalCrossentropy(from_logits=False), activation='relu', lr=0.001):
    model_classifier = tf.keras.models.Sequential([
      keras.layers.Conv2D(32, [3, 3], activation=activation, input_shape=(28, 28, 1)),
      keras.layers.Conv2D(64, [3, 3], activation=activation),
      keras.layers.MaxPooling2D(pool_size=(2, 2)),
      keras.layers.Dropout(0.25),
      keras.layers.Flatten(),
      keras.layers.Dense(128, activation=activation),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(10, activation='softmax')
    ])

    model_extractor = tf.keras.Model(inputs=model_classifier.input,
                                     outputs=model_classifier.layers[-3].output)
    model_extractor.compile(run_eagerly=True)
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model_classifier.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[tf.metrics.CategoricalAccuracy()],
                  run_eagerly=True)

    return model_extractor, model_classifier

def train_model(model, checkpoint_path, restore='', epochs=5):
    checkpoint_dir = os.path.dirname(checkpoint_path)

    if restore:
        model.load_weights(restore)
        print('Restored model from {}'.format(restore))

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path+"ck-{epoch:02d}", verbose=0,
        save_weights_only=True)
    callbacks = []

    if args.save_checkpoint:
        callbacks.append(cp_callback)
    if args.rate!=0:
        keras.backend.set_value(model.optimizer.learning_rate, args.rate)

    # Train the model with the new callback 
    model.fit(train_images,
          train_labels,
          batch_size=128,
          epochs=epochs,
          validation_data=(test_images, test_labels),
          callbacks=callbacks) # Pass callback to training

In [5]:
# Create a basic model instance
loss_func = tf.losses.CategoricalCrossentropy(from_logits=False)
if args.loss == '' or args.loss == 'entropy':
    loss_func = tf.losses.CategoricalCrossentropy(from_logits=False)
elif args.loss == 'faulty':
    print("Using faulty loss")
    loss_func = faultyloss
elif args.loss == 'mse':
    loss_func = tf.losses.MeanSquaredError()
extractor, model = create_model(loss=loss_func, activation=args.activation, lr=args.rate)
result = train_model(model, args.path+'/', restore=args.checkpoint, epochs=args.epoch)

test_features = []
# for i in range(int(test_images.shape[0]/1000)):
for i in range(1):
    print('Processed ', i*100, ' samples')
    test_features.append(extractor.predict(test_images[i*100:i*100+100]))
test_features = tf.concat(test_features, 0)

train_features = []
train_features = []
for i in range(int(train_images.shape[0]/1000)):
    print('Processed ', i*100, ' samples')
    train_features.append(extractor.predict(train_images[i*100:i*100+100]))
train_features = tf.concat(train_features, 0)

/home/wp2280/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


469/469 [==============================] - 25s 54ms/step - loss: 0.2377 - categorical_accuracy: 0.9282 - val_loss: 0.0437 - val_categorical_accuracy: 0.9863
Processed  0  samples
Processed  0  samples


In [6]:
# Evaluate value
from ShapEngine import ShapEngine

test_features = test_features[:20]
train_features = train_features[:20]
test_features_np = test_features.numpy()
train_features_np = train_features.numpy()
test_labels_np = test_labels.numpy()[0:20]
train_labels_np = train_labels.numpy()[0:20]
train_labels_np = np.argmax(train_labels_np, axis=1)
test_labels_np = np.argmax(test_labels_np, axis=1)

model_family='logistic'
metric='accuracy'
GR_threshold=1.05
weights_list=[(1, 16), (1, 4), (1,1), (4,1), (16, 1)]

shap_engine=ShapEngine(X=train_features_np, y=train_labels_np, X_val=test_features_np, y_val=test_labels_np, 
                       problem='classification', model_family=model_family, 
                       metric=metric, GR_threshold=GR_threshold)
%time shap_engine.run(weights_list=weights_list)

(20, 128)
(20, 128)
(20,)
(20,)
Source is initialized. A unit of sample is one data point
Start: Marginal Contribution Calculation!
Here
Here
Here
Here


In [ ]:
# A vector of data values is stored in `shap_engine.results` 
print(f'List of data values: {list(shap_engine.results.keys())}')

List of data values: ['Beta(16,1)', 'Beta(4,1)', 'Beta(1,1)', 'Beta(1,4)', 'Beta(1,16)', 'LOO-Last']


In [ ]:
# Beta(4:1) data values for the first 10 data points. 
first_ten_values=shap_engine.results['Beta(4,1)'][:10]
print(f'First 10 data values: {first_ten_values}')

First 10 data values: [0.0288301  0.01684534 0.02828557 0.0250167  0.02825778 0.02424356
 0.02590261 0.02547816 0.02338693 0.0285735 ]


In [ ]:
MC_mat=shap_engine.MC_obs_card_mat/(shap_engine.MC_count_obs_card_mat+1e-16)
print(f'Shape of MC_mat : {MC_mat.shape}')
print(f'First 12 marginal contributions of the first sample:\n {np.array(MC_mat[0])[:12]}')

Shape of MC_mat : (100, 100)
First 12 marginal contributions of the first sample:
 [ 0.00000000e+00  3.46944695e-18  6.08333333e-02  6.50000000e-02
 -1.00000000e-02  4.66666667e-02  5.71428571e-02  4.30769231e-02
  4.20000000e-02  4.75000000e-02  3.40000000e-02  2.75000000e-02]


# Marginal contributions for clean and noisy samples
- Figure 2 in the manuscript shows a smooth curve because it is based on 50 independent runs.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
clean_index=[i for i in range(MC_mat.shape[0]) if i not in noisy_index]
plt.plot(np.arange(MC_mat.shape[0]), np.mean(MC_mat[clean_index], axis=0), label='Clean')
plt.plot(np.arange(MC_mat.shape[0]), np.mean(MC_mat[noisy_index], axis=0), label='Noisy')
plt.legend(fontsize=15)
plt.xlabel('Cardinality', fontsize=15)
plt.ylabel('Marginal Contributions', fontsize=15)

NameError: name 'noisy_index' is not defined

# Performance on downstream ML tasks

In [ ]:
%time result_dict=utils.summary_experiments(shap_engine, noisy_index, X_test, y_test)
print(f'Available ML tasks: {list(result_dict.keys())}')

CPU times: user 20.7 s, sys: 286 ms, total: 21 s
Wall time: 3.58 s
Available ML tasks: ['noisy', 'subsampling', 'point_removal', 'point_addition']


In [ ]:
# For instance, a noisy label detection task result is stored in result_dict['noisy'] 
print('For each method, the following dictionary shows recall, precision, and F1-score.')
result_dict['noisy'] 

For each method, the following dictionary shows recall, precision, and F1-score.


{'Beta(16,1)': [0.35, 0.3888888888888889, 0.3684210526315789],
 'Beta(4,1)': [0.45, 0.45, 0.44999999999999996],
 'Beta(1,1)': [0.5, 0.35714285714285715, 0.4166666666666667],
 'Beta(1,4)': [0.35, 0.14285714285714285, 0.20289855072463767],
 'Beta(1,16)': [0.3, 0.11764705882352941, 0.16901408450704225],
 'LOO-Last': [0.15, 0.0625, 0.08823529411764705]}